<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
import RBmodel

from RBmodel import cell_models

from RBmodel.plot_utils import plot_phase_space

import numpy as np

import matplotlib.pyplot as plt

import os

In [ ]:
def get_last_phase(cell):
    phases = np.array(cell.phase_hist)
    
    phases = (phases == "G1").astype(int)
    
    dphases = phases[1:] - phases[:-1]
    
    G1idx = np.where(dphases==1.)[0]
    
    start, end = G1idx[-2]+1, G1idx[-1]-1 # indices of the last phase
    
    G2idx = np.where(dphases==-1.)[0]
    
    G1S_trans = G2idx[((G2idx > start) & (G2idx < end))][0]
    
    t = np.linspace(start-G1S_trans, end-G1S_trans, end-start)*params['dt']
    return start, end, t

In [ ]:
save = False

# General model

In [ ]:
params = cell_models.DEFAULT_PARAMS

In [ ]:
params['delta'] = .8
params['transition'] = 'size'
params['k_trans'] = 1000
params['alpha'] = .1
params['eta'] = 1.2


In [ ]:
params

In [ ]:
T = int(1e5)

In [ ]:
cell = cell_models.cell()

cell.grow(T)

In [ ]:
start, end, t = get_last_phase(cell)

In [ ]:
SAVE_DIR = "/Users/shuyuan/Library/CloudStorage/OneDrive-Stanford/Skotheim lab/RB degradation paper/Figures"

In [ ]:
yvals = cell.RB_c_hist

plt.plot(t, yvals[start:end])

if save:
    plt.savefig(
        os.path.join(SAVE_DIR, "Figure 1", "RBc.eps"), dpi=300
    )

In [ ]:
yvals = cell.RB_hist

plt.plot(t, yvals[start:end])

if save:
    plt.savefig(
        os.path.join(SAVE_DIR, "Figure 1", "RB_amount.eps"), dpi=300
    )

# Comparing with dilution-only model

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(6, 3))
y0 = np.array(cell.RB_c_hist[start:end])
y1 = np.array(cell.RB_hist[start:end])
M_vec = np.array(cell.M_hist[start:end])

ax[0].plot(t, y0, label="Current model")
ax[0].plot(t, y1[0]/M_vec, label="Dilution model")
ax[0].set_xlim(t[0], 0)

ax[1].plot(t, y1, label="Current model")
ax[1].axhline([y1[0]], label="Dilution model", color="darkorange")
plt.xlim(t[0], 0)

# plt.axvline([0], linestyle="--", color="grey")

# plt.axvline([0], linestyle="--", color="grey")

ax[0].legend()
ax[1].legend()

ax[0].set_xlabel("Time [hrs]")
ax[1].set_xlabel("Time [hrs]")
ax[0].set_ylabel("RB concentration")
ax[1].set_ylabel("RB amount")

fig.tight_layout()

ax[0].set_ylim(1, 2)
ax[1].set_ylim(1, 1.5)

# Comparing the contribution of dilution vs degradation

Here we compare the concentration trends under three different models:
- the original model, that contains both degradation and dilution
- an alternative model with only degradation: this amounts to looking at the RB amounts curve from the initial model, and assuming that the mass is constant
- an alternative model where the degradation rate is zero: in this case we only look at the contribution of dilution

In [ ]:
params2 = cell_models.DEFAULT_PARAMS

In [ ]:
params2['delta'] = .8
params2['transition'] = 'size'
params2['k_trans'] = 1000
params2['alpha'] = .1
params2['eta'] = 1.2
params2['beta0'] = 0.


In [ ]:
params2

In [ ]:
T = int(1e5)

In [ ]:
cell2 = cell_models.cell()

cell2.grow(T)

In [ ]:
start2, end2, t2 = get_last_phase(cell2)

In [ ]:
yvals2 = cell2.RB_c_hist

plt.plot(t2, yvals2[start2:end2])


In [ ]:
y0 = np.array(cell.RB_c_hist[start:end])
y1 = np.array(cell.RB_hist[start:end])
y2 = np.array(cell2.RB_c_hist[start2:end2])

plt.plot(t, y0/y0[0], label="Degradation + dilution")
plt.plot(t, y1/y1[0], label="Degradation only")
plt.plot(t2, y2/y2[0], label="Dilution only")
plt.ylim(.5, 1.1)

plt.axvline([0], linestyle="--", color="grey")

plt.legend()

In [ ]:
M_vec = np.array(cell.M_hist[start:end])
plt.plot(t, M_vec/M_vec[0])
plt.grid()
plt.ylabel("M")
plt.xlabel("t")

In [ ]:
1- 1/1.4